In [1]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import os

# 예제 011의 구글 뉴스 클리핑 함수를 다시 사용    
def google_news_clipping_keyword(keyword_input='파이썬', limit=5):
    
    keyword = urllib.parse.quote(keyword_input)
    
    base_url = "https://news.google.com"
    url = base_url + "/search?q=" + keyword + "&hl=ko&gl=KR&ceid=KR%3Ako"
    
    resp = requests.get(url)
    html_src = resp.text
    soup = BeautifulSoup(html_src, 'lxml')
    
    news_items = soup.select('div[class="xrnccd"]')
    
    links=[]; titles=[]; contents=[]; agencies=[]; reporting_dates=[]; reporting_times=[];
    
    for item in news_items[:limit]:
        link = item.find('a', attrs={'class':'VDXfz'}).get('href')
        news_link = base_url + link[1:]
        links.append(news_link)
        
        news_title = item.find('a', attrs={'class':'DY5T1d'}).getText()
        titles.append(news_title)
    
        news_content = item.find('span', attrs={'class':'xBbh9'}).text
        contents.append(news_content)
    
        news_agency = item.find('a', attrs={'class':'wEwyrc AVN2gc uQIVzc Sksgp'}).text
        agencies.append(news_agency)
    
        news_reporting = item.find('time', attrs={'class':'WW6dff uQIVzc Sksgp'})
        news_reporting_datetime = news_reporting.get('datetime').split('T')
        news_reporting_date = news_reporting_datetime[0]
        news_reporting_time = news_reporting_datetime[1][:-1]
        reporting_dates.append(news_reporting_date)
        reporting_times.append(news_reporting_time)     
    
    result = {'link':links, 'title':titles, 'contents':contents, 'agency':agencies, \
              'date':reporting_dates, 'time':reporting_times}
    
    return result

# 함수를 실행하여 뉴스 목록 정리
search_word = input('검색어를 입력하세요:  ')
news =google_news_clipping_keyword(search_word, 5)
df = pd.DataFrame(news)
print(df.head())

filepath = os.path.join(os.getcwd(), 'output', 'google_news_to_dataframe_excel.xlsx')
df.to_excel(filepath)

검색어를 입력하세요:  엑소
                                                link  \
0  https://news.google.com/articles/CBMiP2h0dHA6L...   
1  https://news.google.com/articles/CBMiP2h0dHA6L...   
2  https://news.google.com/articles/CBMiPWh0dHA6L...   
3  https://news.google.com/articles/CBMiOWh0dHA6L...   
4  https://news.google.com/articles/CBMiPWh0dHA6L...   

                                               title  \
0  엑소 찬열, 소통지수 크게 하락하며 세 계단 내려와...2020년 8월 보이그룹 개...   
1  엑소, 평판지수 상승하며 네 계단 올라서...2020년 7월 아이돌그룹 100대 브...   
2     엑소 찬열, 광복절 일본 노래 커버 논란…"다른 날도 아니고 굳이" - 강소현 기자   
3                 : 블랙핑크, '최애돌' 기부천사로 선정…엑소는 17개월 연속   
4  엑소(EXO)-블랙핑크(BLACKPINK), '최애돌' 제59대 기부천사 선정…가장...   

                                            contents    agency        date  \
0  엑소 찬열이 총 브랜드평판지수 하락률을 보이며, 지난 7월에 비해 순위 세 계단 하...  미래 한국 신문  2020-08-18   
1  엑소가 총 브랜드평판지수 상승률을 보이며, 아이돌그룹 100대 브랜드평판 8위를 차...  미래 한국 신문  2020-07-28   
2  엑소(EXO) 찬열이 광복절인 15일 일본 가수 노래를 커버해 사운드클라우드에 올렸...     톱스타뉴스  2020-08-16  